<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [77]:
import pandas as pd

df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv", na_values=' ')

In [58]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [78]:
# remove customerID
df = df.drop(["customerID"], axis=1)

In [79]:
# Remove missing values from TotalCharges
df = df.dropna()

In [80]:
# Turn yes/no into booleans
yesNoD = {"Yes": 1, "No": 0}
yesNoColumns = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

for c in yesNoColumns:
    df[c] = df[c].replace(yesNoD)

In [81]:
import category_encoders as ce

# One-Hot encode categorical data
df = ce.OneHotEncoder().fit_transform(df)

In [82]:
df.head()

,gender_1,gender_2,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines_1,MultipleLines_2,MultipleLines_3,...,Contract_2,Contract_3,PaperlessBilling,PaymentMethod_1,PaymentMethod_2,PaymentMethod_3,PaymentMethod_4,MonthlyCharges,TotalCharges,Churn
0,1,0,0,1,0,1,0,1,0,0,...,0,0,1,1,0,0,0,29.85,29.85,0
1,0,1,0,0,0,34,1,0,1,0,...,1,0,0,0,1,0,0,56.95,1889.50,0
2,0,1,0,0,0,2,1,0,1,0,...,0,0,1,0,1,0,0,53.85,108.15,1
3,0,1,0,0,0,45,0,1,0,0,...,1,0,0,0,0,1,0,42.30,1840.75,0
4,1,0,0,0,0,2,1,0,1,0,...,0,0,1,1,0,0,0,70.70,151.65,1


In [89]:
from sklearn.preprocessing import MinMaxScaler

# Scale data
mms = MinMaxScaler()

X = mms.fit_transform(df.drop(['Churn'], axis=1))
y = df['Churn'].values

C:\Users\soobt\Anaconda3\envs\jupyterlab\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [138]:
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam


def create_model(learning_rate=0.001, dropout_rate=0.2, layer1_size = 30, activation1='relu'):
    adam = Adam(learning_rate=learning_rate)
    
    # create model
    model = Sequential()
    model.add(Dense(layer1_size, input_dim=41, activation=activation1))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

In [152]:
baseline = create_model()
baseline.fit(X, y, epochs=100, validation_split=0.2, verbose=0)

In [153]:
import numpy as np
from sklearn.metrics import accuracy_score
print(f"Accuracy: {accuracy_score(np.round(baseline.predict(X)),y)}")

Accuracy: 0.820250284414107


In [131]:
param_grid = {'layer1_size': [10] # [10,15,20,25,30,35,40,45,50,55,60,65,70,75,80]
             ,'activation1' : ['relu'] # ['relu', 'softmax', 'selu', 'tanh', 'sigmoid', 'hard_sigmoid', 'exponential', 'linear', 'elu']
             ,'batch_size': [10] # [10, 20, 40, 60, 80, 100]
             ,'dropout_rate' : [0.2] # [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
             ,'learning_rate' : [0.001] # [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
             ,'epochs': [10] # [20, 40, 60, 200]
#             , 'beta_1' : [.9]
#             , 'beta_2' : [.999]
              , 
             }

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(X, y)

In [132]:
param_grid = {'layer1_size': [10,15,20,25,30,35,40,45,50,55,60,65,70,75,80]
             ,'activation1' : ['relu'] # ['relu', 'softmax', 'selu', 'tanh', 'sigmoid', 'hard_sigmoid', 'exponential', 'linear', 'elu']
             ,'batch_size': [10] # [10, 20, 40, 60, 80, 100]
             ,'dropout_rate' : [0.2] # [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
             ,'learning_rate' : [0.001] # [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
             ,'epochs': [10] # [20, 40, 60, 200]
             }

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4, cv=5)
grid_result = grid.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.8037542544720658 using {'activation1': 'relu', 'batch_size': 10, 'dropout_rate': 0.2, 'epochs': 10, 'layer1_size': 55, 'learning_rate': 0.001}


In [133]:
param_grid = {'layer1_size': [55]
             ,'activation1' : ['relu', 'softmax', 'selu', 'tanh', 'sigmoid', 'hard_sigmoid', 'exponential', 'linear', 'elu']
             ,'batch_size': [10] # [10, 20, 40, 60, 80, 100]
             ,'dropout_rate' : [0.2] # [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
             ,'learning_rate' : [0.001] # [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
             ,'epochs': [10] # [20, 40, 60, 200]
             }

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4, cv=5)
grid_result = grid.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.8040386708930084 using {'activation1': 'tanh', 'batch_size': 10, 'dropout_rate': 0.2, 'epochs': 10, 'layer1_size': 55, 'learning_rate': 0.001}


In [134]:
param_grid = {'layer1_size': [55]
             ,'activation1' : ['tanh']
             ,'batch_size': [10, 20, 40, 60, 80, 100]
             ,'dropout_rate' : [0.2] # [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
             ,'learning_rate' : [0.001] # [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
             ,'epochs': [10] # [20, 40, 60, 200]
             }

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4, cv=5)
grid_result = grid.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.8037542778579041 using {'activation1': 'tanh', 'batch_size': 10, 'dropout_rate': 0.2, 'epochs': 10, 'layer1_size': 55, 'learning_rate': 0.001}


In [135]:
param_grid = {'layer1_size': [55]
             ,'activation1' : ['tanh']
             ,'batch_size': [10]
             ,'dropout_rate' : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
             ,'learning_rate' : [0.001] # [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
             ,'epochs': [10] # [20, 40, 60, 200]
             }

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4, cv=5)
grid_result = grid.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.8051763328642026 using {'activation1': 'tanh', 'batch_size': 10, 'dropout_rate': 0.3, 'epochs': 10, 'layer1_size': 55, 'learning_rate': 0.001}


In [137]:
param_grid = {'layer1_size': [55]
             ,'activation1' : ['tanh']
             ,'batch_size': [10]
             ,'dropout_rate' : [0.3]
             ,'learning_rate' : [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
             ,'epochs': [100]
             }

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=4, cv=5)
grid_result = grid.fit(X, y)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.8027588248812297 using {'activation1': 'tanh', 'batch_size': 10, 'dropout_rate': 0.3, 'epochs': 100, 'layer1_size': 55, 'learning_rate': 0.0001}


In [156]:
HOModel = create_model(activation1='tanh', dropout_rate=0.3, layer1_size=55, learning_rate=0.0001)
HOModel.fit(X, y, epochs=100, batch_size=10, validation_split=0.2, verbose=0)

In [157]:
print(f"Accuracy: {accuracy_score(np.round(HOModel.predict(X)),y)}")

Accuracy: 0.8054607508532423


Amazing. The hyperparameter optimization did NOTHING. In fact, it made things worse!

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?